In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
from google.colab import drive

ModuleNotFoundError: No module named 'google.colab'

Load/Examine Data

In [3]:
#preprocess data
file_path = "GPT-wiki-intro.csv"
df = pd.read_csv(file_path, error_bad_lines = False, engine = "python")

C:\Users\jzasl\AppData\Local\Temp\ipykernel_4220\3754555685.py:3: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv(file_path, error_bad_lines = False, engine = "python")


In [4]:
#Explore dataset
print(df.shape)
print(df.columns)

(150000, 12)
Index(['id', 'url', 'title', 'wiki_intro', 'generated_intro', 'title_len',
       'wiki_intro_len', 'generated_intro_len', 'prompt', 'generated_text',
       'prompt_tokens', 'generated_text_tokens'],
      dtype='object')


In [5]:
df.head

<bound method NDFrame.head of               id                                                url  \
0       63064638  https://en.wikipedia.org/wiki/Sexhow%20railway...   
1         279621        https://en.wikipedia.org/wiki/Eti%C3%A4inen   
2         287229  https://en.wikipedia.org/wiki/Inverse%20functi...   
3       26712375  https://en.wikipedia.org/wiki/Stepping%20on%20...   
4       38894426        https://en.wikipedia.org/wiki/Rob%20Bradley   
...          ...                                                ...   
149995  44173767        https://en.wikipedia.org/wiki/Randy%20Borum   
149996  33564134  https://en.wikipedia.org/wiki/Sa%27och%20language   
149997   4219548    https://en.wikipedia.org/wiki/Philip%20Hanawalt   
149998   2625970  https://en.wikipedia.org/wiki/Vossius%20Gymnasium   
149999  32345102    https://en.wikipedia.org/wiki/Simone%20Stratigo   

                           title  \
0         Sexhow railway station   
1                       Etiäinen   
2       I

Create Train/Test Split (pyCaret Does Validation)

In [6]:
#Explore dataset
train_test_split = 0.75

human = df.copy()
gpt = df.copy()

human['text'] = df['wiki_intro']
gpt['text'] = df['generated_intro']

human = human[['text']]
gpt = gpt[['text']]

human = human.iloc[::2, :]
gpt = gpt.iloc[1::2, :]

#add label for train
human['label'] = 1.0
gpt['label'] = 0.0

#stack dataframes and shuffle to create train set
from sklearn.utils import shuffle

dataset = pd.concat([human, gpt], ignore_index=True)
#shuffle dataset for later batch processing
dataset = shuffle(dataset)

train = dataset.iloc[:int(np.floor(df.shape[0] * train_test_split))]

#subtract 2 to prevent train/test independence violation
test = dataset.tail(int(np.floor(df.shape[0] * (1 - train_test_split) - 2)))

print(train.head)

<bound method NDFrame.head of                                                      text  label
17813   Joseph Kalimbwe (born 25 March 1993) is a Zamb...    1.0
88230   Kandavas Kartodroms is a karting circuit in th...    0.0
22038   Shipley Hollow Road is a road in Sale Creek, T...    1.0
50812   On June 16, 2015, shortly after midnight, five...    1.0
40608   A passenger railroad car or passenger car (Uni...    1.0
...                                                   ...    ...
82626   Makatea, or Mangaia-te-vai-tamae, is a raised ...    0.0
136261  Salif Diallo (9 May 1957 – 19 February 1999) w...    0.0
111634  Iosif "Sifis" Daskalakis (; born 7 August 1967...    0.0
147692  Boone County Airlines was a passenger airline ...    0.0
104355  Phyllis Spira (18 October 1943 – 11 May 2002) ...    0.0

[112500 rows x 2 columns]>


Preprocess Data for BERT

In [7]:
#copied from kaggle https://www.kaggle.com/code/oldjerry/multi-model-fusion-based-on-baye
import re
import nltk
import tensorflow as tf
from nltk.corpus import wordnet
from nltk import word_tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tqdm
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

def removeHTML(x):
    html=re.compile(r'<.*?>')
    return html.sub(r'',x)


# lemm = nltk.stem.WordNetLemmatizer()
# w_tokenizer = nltk.tokenize.WhitespaceTokenizer()

# def get_wordnet_pos(word):
#     #Map POS tag to first character lemmatize() accepts
#     tag = nltk.pos_tag([word])[0][1][0].upper()
#     tag_dict = {"J": wordnet.ADJ,
#                 "N": wordnet.NOUN,
#                 "V": wordnet.VERB,
#                 "R": wordnet.ADV}

#     return tag_dict.get(tag, wordnet.NOUN)

# def lemmatize(text):
#     return [lemm.lemmatize(w, get_wordnet_pos(w)) for w in w_tokenizer.tokenize(text)]

def dataPreprocessing(x):    
    x = x.lower()
    x = removeHTML(x)
    x = re.sub("@\w+", '',x) # removing mentions (@)
    x = re.sub("'\d+", '',x)
    x = re.sub("\d+", '',x)
    x = re.sub(r"[^\w\s]", '',x) # to remove symbols
    x = re.sub("http\w+", '',x)
    x = re.sub("\s[a-z]\s", '',x)
    x = x.strip()
    return x

train_no_delim = train["text"].apply(lambda x: dataPreprocessing(x))
test_no_delim = test["text"].apply(lambda x: dataPreprocessing(x))

train["text"] = train_no_delim
test["text"] = test_no_delim

train_tokens = train_no_delim.apply(lambda x: word_tokenize(x))
test_tokens = test_no_delim.apply(lambda x: word_tokenize(x))

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_tokens)
train_sequences = tokenizer.texts_to_sequences(train_tokens)
test_sequences = tokenizer.texts_to_sequences(test_tokens)

maxlen = max([len(seq) for seq in train_sequences])
train_sequences = pad_sequences(train_sequences, maxlen=maxlen)
test_sequences = pad_sequences(test_sequences, maxlen=maxlen)


print(train_sequences[0])
train_labels = train["label"]
test_labels = test["label"]

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jzasl\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jzasl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
C:\Users\jzasl\AppData\Local\Temp\ipykernel_4220\1050673859.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train["text"] = train_no_delim
C:\Users\jzasl\AppData\Local\Temp\ipykernel_4220\1050673859.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docu

[     0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      0      0      0      0      0      0      0      0      0      0
      

In [8]:
print(test.shape)
train.head
train.columns
print(train.head)
print(test.head)
print(tf.test.is_gpu_available())

(37498, 2)
<bound method NDFrame.head of                                                      text  label
17813   joseph kalimbwe born  march  iszambian politic...    1.0
88230   kandavas kartodroms iskarting circuit in the c...    0.0
22038   shipley hollow road isroad in sale creek tenne...    1.0
50812   on june   shortly after midnight five irishvis...    1.0
40608   a passenger railroad car or passenger car unit...    1.0
...                                                   ...    ...
82626   makatea or mangaiatevaitamae israised coral at...    0.0
136261  salif diallo  may    february  wassenegalese p...    0.0
111634  iosif sifis daskalakis  born  august  isgreek ...    0.0
147692  boone county airlines waspassenger airline bas...    0.0
104355  phyllis spira  october    may  was an australi...    0.0

[112500 rows x 2 columns]>
<bound method NDFrame.head of                                                      text  label
81345   oligodon churahensis the churah valley kukri s..

 Extract Additional Features from Text to Add to Data

In [9]:
#create basic tensorflow RNN
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, CuDNNGRU, CuDNNLSTM, BatchNormalization, Bidirectional

train_sequences = np.array(train_sequences)
test_sequences = np.array(test_sequences)

input_shape = (maxlen,)

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=128, input_shape=input_shape),
    tf.compat.v1.keras.layers.CuDNNLSTM(64, return_sequences=True),
    tf.keras.layers.Dropout(0.5),
    tf.compat.v1.keras.layers.CuDNNLSTM(32),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

model.fit(train_sequences, train_labels, epochs=10, batch_size=128)
loss, accuracy = model.evaluate(test_sequences, test_labels, verbose=1)
print(f'Test accuracy: {accuracy:.4f}')



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 408, 128)          73161088  
                                                                 
 cu_dnnlstm (CuDNNLSTM)      (None, 408, 64)           49664     
                                                                 
 dropout (Dropout)           (None, 408, 64)           0         
                                                                 
 cu_dnnlstm_1 (CuDNNLSTM)    (None, 32)                12544     
                                                                 
 dropout_1 (Dropout)         (None, 32)                0         
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 73,223,329
Trainable params: 73,223,329
N

InvalidArgumentError: No OpKernel was registered to support Op 'CudnnRNNV2' used by {{node sequential/cu_dnnlstm/CudnnRNNV2}} with these attrs: [dropout=0, seed=0, input_mode="linear_input", T=DT_FLOAT, direction="unidirectional", rnn_mode="lstm", is_training=true, seed2=0]
Registered devices: [CPU]
Registered kernels:
  <no registered kernels>

	 [[sequential/cu_dnnlstm/CudnnRNNV2]] [Op:__inference_train_function_3019]

In [ ]:
import nltk
from nltk import word_tokenize
from nltk.lm.preprocessing import padded_everygram_pipeline
from nltk.lm import MLE
from collections import Counter

nltk.download('punkt')

import nltk
from nltk.lm import MLE
from nltk.lm.preprocessing import padded_everygram_pipeline

# Create a language model using the training dataset
n = 4
train_data = train[train['label'] == 1]
tokens = nltk.word_tokenize(' '.join( train_data["text"] ))
train_data, padded_vocab = padded_everygram_pipeline(n, tokens)
model = MLE(n)
model.fit(train_data, padded_vocab)


In [ ]:

def extract_perplexity(dataset, model, n):
    """
    Extract the perplexity of each text sample in the given dataset.
    takes a pd returns ped with perplexity column
    """
    # Calculate the perplexity of each text sample using the model
    perplexities = []
    for text in dataset['text']:
        tokens = nltk.word_tokenize(text)
        test_data, _ = padded_everygram_pipeline(n, tokens[:-1])
        test_text = tokens[-1]
        perplexity = model.perplexity(test_data)
        perplexities.append(perplexity)
    
    # Add the perplexity column to the dataset
    dataset['perplexity'] = perplexities
    
    return dataset


def avg_burstiness(text):
    """
    Computes the avg burstiness of given text
    GPT-0 inspired
    """
    # Split the text into words
    words = text.split()
    word_counts = Counter(words)
    mean_frequency = sum(word_counts.values()) / len(word_counts)
    variance = sum((count - mean_frequency) ** 2 for count in word_counts.values()) / len(word_counts)
    burstiness_scores = [(count - mean_frequency) / variance for count in word_counts.values()]
    average_burstiness = sum(burstiness_scores) / len(burstiness_scores)
    return average_burstiness

def avg_sentence_length(text):
    """
    Calculates the average length of sentences in the given text
    AI may have more standard sentence length with less variation
    """
    sentences = re.split(r'[.?!]', text)
    lengths = [len(word_tokenize(sentence)) for sentence in sentences if len(sentence) > 0]
    return sum(lengths) / len(lengths)

def vocabulary_richness(text):
    """
    Calculates the ratio of unique words to total words in the given text.
    AI may have less rich vocabulary in its text
    """
    tokens = word_tokenize(text.lower())
    unique_words = set(tokens)
    return len(unique_words) / len(tokens)

extract_feature_functions = [ avg_burstiness, avg_sentence_length, vocabulary_richness]

#we can keep trying to extract features that might be useful from the text 
# def count_contractions(text):
#     """
#     Counts the number of contractions in the given text.
#     """
#     contractions = ["'s", "'m", "'ve", "'re", "'d", "n't", "'ll"]
#     tokens = word_tokenize(text.lower())
#     count = sum([1 for token in tokens if token in contractions])
#     return count




In [ ]:

##add these features to train and test sets 
train = extract_perplexity(train, model, n)
test = extract_perplexity(test, model, n)
for func in extract_feature_functions: 
  train[func.__name__] = train['text'].apply(func)
  test[func.__name__] = test['text'].apply(func)




Get BERT Encodings

In [ ]:
# Load torch
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(torch.cuda.is_available())
print(device)

In [ ]:
# tokenize words
import nltk
nltk.download('stopwords')

from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, f1_score
from scipy.sparse import hstack
stopwords = stopwords.words('english')
stopwords = stopwords[:116]
stopwords = stopwords.extend(['d', 'll', 're', 's', 've'])
data = pd.concat([train_no_delim, test_no_delim],axis=0).reset_index()["text"]
data

In [ ]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words=stopwords,
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(data)
train_word_features = word_vectorizer.transform(train_no_delim)
test_word_features = word_vectorizer.transform(test_no_delim)

In [ ]:
train_vars = hstack([train_word_features])
test_vars = hstack([test_word_features])

In [ ]:
from torch import nn
!pip install transformers

from transformers import RobertaTokenizer, RobertaModel
from transformers import AdamW
from tqdm import tqdm
class BertClassificationModel(nn.Module):
    def __init__(self,hidden_size=768,num_class=2): 
        super(BertClassificationModel, self).__init__()
        model_name = "roberta-base"
        self.tokenizer = RobertaTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
        self.bert = RobertaModel.from_pretrained(pretrained_model_name_or_path=model_name)

        for p in self.bert.parameters(): 
                p.requires_grad = False
        self.fc = nn.Linear(hidden_size, num_class)

    def forward(self, batch_sentences): 
        sentences_tokenizer = self.tokenizer(batch_sentences,
                                             truncation=True,
                                             padding=True,
                                             max_length=512,
                                             add_special_tokens=True)
        input_ids=torch.tensor(sentences_tokenizer['input_ids']).to(device) 
        attention_mask=torch.tensor(sentences_tokenizer['attention_mask']).to(device) 
        bert_out=self.bert(input_ids=input_ids,attention_mask=attention_mask) 

        last_hidden_state =bert_out[0].to(device) 
        bert_cls_hidden_state=last_hidden_state[:,0,:].to(device) 
        return bert_cls_hidden_state
model=BertClassificationModel()
model=model.to(device)

In [ ]:
# reform for torch model
torch_train = []
for i in tqdm(train_no_delim):
    torch_train.append(model([i]).cpu().detach().numpy())
torch_train = np.array(torch_train).reshape(-1,768)  
del train_no_delim

In [ ]:
torch_test = []
for i in tqdm(test_no_delim):
    torch_test.append(model([i]).cpu().detach().numpy())
torch_test = np.array(torch_test).reshape(-1,768)
del test_no_delim

Save Test and Training Set

In [ ]:

##first test a model with just the BERT features
##add Bert encodings to dataset with new features

train["Bert"] = torch_train
test["Bert"] = torch_test

#write to file so can load later 
train.to_pickle('train.pkl')
test.to_pickle('test.pkl')

#load data
train = pd.read_pickle('train.pkl')
test = pd.read_pickle('test.pkl')

PyCaret To Find Best Classifier Model

In [ ]:
# !pip install pycaret
import pycaret
from pycaret.classification import *

# initialize the PyCaret environment
setup(train, target="label", silent=True)

# compare and tune multiple models
best_model = compare_models()

# print the best model's hyperparameters
print(best_model)

Evaluate Best Classifier on Test Set

In [ ]:
from pycaret.classification import *
predictions = predict_model(best_model, data=test_data)
print(classification_report(predictions['label'], predictions['Label'])

Create Maybe 100 AI Generated Product Reviews using ChatGPT -- Test Performance of Model

In [ ]:
amazon_products = ['Echo Dot (4th Gen) Kids', 'Samsung Galaxy S21 5G', 'Apple AirPods Pro', 'Fujifilm Instax Mini 11', 'Bose QuietComfort 35 II Wireless Bluetooth Headphones', 'Kindle Oasis - Now with adjustable warm light', 'Anker PowerCore 10000 Portable Charger', 'Fitbit Charge 4 Fitness and Activity Tracker', 'Wemo Smart Light Switch', 'Canon EOS Rebel T7 DSLR Camera', 'Instant Pot Duo Plus 9-in-1 Electric Pressure Cooker', 'Apple iPad (8th Generation) Tablet', 'Beats Solo Pro Wireless Noise Cancelling On-Ear Headphones', 'Samsung UN55RU7300FXZA Curved 55-Inch 4K UHD 7 Series Ultra HD Smart TV', 'Nintendo Switch Lite', 'Ninja Foodi 9-in-1 Pressure, Slow Cooker, Air Fryer and More', 'Ring Video Doorbell 3 with Echo Show 5', 'Tile Mate (2020) Bluetooth Item Finder', 'YETI Rambler 20 oz Tumbler', 'Sony WH-1000XM4 Wireless Noise Cancelling Headphones', 'Keurig K-Elite Coffee Maker', 'Breville BES870XL Barista Express Espresso Machine', 'Dyson Cyclone V10 Absolute Lightweight Cordless Stick Vacuum Cleaner', 'Nikon D3500 DSLR Camera', 'Fitbit Versa 2 Health and Fitness Smartwatch', 'Philips Sonicare DiamondClean Smart 9300 Electric Toothbrush', 'Garmin Venu Sq Music, GPS Smartwatch', 'Samsung Galaxy Watch Active2', 'LEGO Star Wars: The Mandalorian The Razor Crest Building Kit', 'All-new Fire HD 8 tablet', 'Instant Pot Duo Nova 7-in-1 Electric Pressure Cooker', 'Apple Watch Series 6', 'Samsung Galaxy Tab A7', 'GoPro HERO9 Black', 'Airpods Case Cover with Keychain for Apple AirPods 2 & 1', 'Oculus Quest 2 All-In-One Virtual Reality Headset', 'Canon PowerShot SX620 Digital Camera', 'Sennheiser Momentum 3 Wireless Noise Cancelling Headphones', 'AmazonBasics Hardside Spinner Luggage', 'Microsoft Surface Laptop 3', 'Jabra Elite 85h Wireless Noise-Canceling Headphones', 'Panasonic Lumix FZ80 4K Digital Camera', 'Ring Video Doorbell 3 Plus', 'Razer Blade 15 Gaming Laptop', 'LG OLED65CXPUA Alexa Built-In CX 65" 4K Smart OLED TV', 'Samsung Galaxy Buds+ Plus', 'Shark Navigator Lift-Away Deluxe NV360 Upright Vacuum', 'Toshiba 
openai.api_key = ""
def gpt_request(prompt):
    max_tokens = 1000
    response = openai.Completion.create(
          model="text-davinci-003",
          prompt=prompt,
          temperature=0.2,
          max_tokens=max_tokens
      )
    print(response)
    return response.choices[0].text
  
def chatgpt_request(prompt):
    response = openai.ChatCompletion.create(
          model="gpt-3.5-turbo-0301",
          messages=[{"role": "user", "content": prompt}, ],
          temperature=0.2,
          # max_tokens=max_tokens
      )
    print(response)
    print(response.choices[0].message.content)
    return response.choices[0].message.content

prompt = "write me an Amazon review for the "
resps = []
for product in amazon_products:
  prompt = prompt + product
  resps.append(gpt_request(prompt))


#throw in some human generated ones and make a labeled dataset


#evaluate model on new product dataset

